$\textbf{AdaBoost分类器：}$

更多地关注前序欠拟合的训练实例，从而使新的预测器不断地越来越专注于难缠的问题。Adaboost不是调整单个预测器的参数使成本函数最小化，而是不断在集成中加入预测器，是模型越来越好。

将每个实例权重$w^{(i)}$最初设置为$\frac{1}{m}$，对第一个预测器进行训练，并根据训练集计算其加权误差率$r_{1}$。

第$j$个预测器的加权误差率：$r_{j}=\frac{\sum^{m}_{i=1,\hat{y}^{(i)}_{j} \neq y^{(i)}} w^{(i)}}{\sum^{m}_{i=1}w^{(i)}} \quad\quad\quad\quad$ $\hat{y}^{(i)}_{j}$是第$i$个实例的第$j$个预测器的预测。

预测器权重：$\alpha_{j}=\eta log\frac{1-r_{j}}{r_{j}}$

权重更新规则：对于$i=1,2,...m \quad\quad$   $
w^{(i)} \leftarrow
\begin{cases}
w^{(i)}, & \text{如果 } \hat{y}_j^{(i)} = y^{(i)} \\
w^{(i)} \exp(\alpha_j), & \text{如果 } \hat{y}_j^{(i)} \neq y^{(i)}
\end{cases}
$，然后对所有实例权重进行归一化。

最后，使用更新后的权重训练一个新的训练器，然后重复整个过程。当到达所需数量的预测器或得到完美的预测器时，算法停止。

Adaboost预测： $\hat{y}(x)=\arg\max_{k} \quad \sum^{N}_{j=1,\hat{y}_{j}(x)=k}\alpha_{j}$

In [3]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
import numpy as np

iris=datasets.load_iris()
X_train=iris["data"][:, (2,3)]
y_train=(iris["target"]==2).astype(np.float64)
ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

,estimator,DecisionTreeC...r(max_depth=1)
,n_estimators,200
,learning_rate,0.5
,algorithm,'deprecated'
,random_state,None
,criterion,'gini'
,splitter,'best'
,max_depth,1
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0


$\textbf{梯度提升}$

In [5]:
from sklearn.tree import DecisionTreeRegressor

X=iris["data"][:, (2,3)]
y=(iris["target"]==2).astype(np.float64)
tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

y2=y-tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

y3=y2-tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

y_pred=sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))

from sklearn.ensemble import GradientBoostingRegressor

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors=[mean_squared_error(y_val, y_pred)
        for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best=GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

gbrt=GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error=float("inf")
error_going_up=0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred=gbrt.predict(X_val)
    val_error=mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error=val_error
        error_going_up=0
    else:
        error_going_up+=1
        if error_going_up==5:
            break #early stopping

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,np.int64(117)
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,2
,min_impurity_decrease,0.0
,init,None


$\textbf{XGboost}$

In [9]:
from xgboost import XGBRegressor
xgb_reg=XGBRegressor(
    n_estimators=1000,
    early_stopping_rounds=2,
    random_state=42,
)

xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)])
y_pred=xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.34390
[1]	validation_0-rmse:0.24413
[2]	validation_0-rmse:0.17340
[3]	validation_0-rmse:0.12430
[4]	validation_0-rmse:0.08970
[5]	validation_0-rmse:0.06467
[6]	validation_0-rmse:0.04707
[7]	validation_0-rmse:0.03390
[8]	validation_0-rmse:0.02460
[9]	validation_0-rmse:0.01804
[10]	validation_0-rmse:0.01346
[11]	validation_0-rmse:0.01003
[12]	validation_0-rmse:0.00751
[13]	validation_0-rmse:0.00565
[14]	validation_0-rmse:0.00428
[15]	validation_0-rmse:0.00325
[16]	validation_0-rmse:0.00248
[17]	validation_0-rmse:0.00190
[18]	validation_0-rmse:0.00146
[19]	validation_0-rmse:0.00113
[20]	validation_0-rmse:0.00087
[21]	validation_0-rmse:0.00068
[22]	validation_0-rmse:0.00052
[23]	validation_0-rmse:0.00041
[24]	validation_0-rmse:0.00035
[25]	validation_0-rmse:0.00029
[26]	validation_0-rmse:0.00026
[27]	validation_0-rmse:0.00022
[28]	validation_0-rmse:0.00020
[29]	validation_0-rmse:0.00019
[30]	validation_0-rmse:0.00018
[31]	validation_0-rmse:0.00018
[32]	validation_0-